In [1]:
import tensorflow as tf
from tensorflow import keras
mnist = tf.keras.datasets.mnist

In [2]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)



Epoch 1/5


1875/1875 [==============================] - 34s 16ms/step - loss: 0.2203 - accuracy: 0.9337
Epoch 2/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0980 - accuracy: 0.9704
Epoch 3/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0671 - accuracy: 0.9790
Epoch 4/5
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0538 - accuracy: 0.9826
Epoch 5/5
313/313 [==============================] - 3s 5ms/step - loss: 0.0632 - accuracy: 0.9814


[0.06324392557144165, 0.9814000129699707]

Is tis architecture optimal?
- Do the model need more or less hidden units to perform well?
- How does model size affect the convergnece speed?
- is there any trade off between coveragence speed, model size and accuracy?
- search automation is the natural path to take
- keras tuner built in search functionality?

In [5]:
# !pip install -q -U keras tuner

In [6]:
import kerastuner as kt

C:\Users\agusa\AppData\Local\Temp\ipykernel_12128\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [14]:
from tensorflow.keras import layers
from kerastuner.tuners import Hyperband
# from kerastuner.tuners import Hyperband

In [32]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    hp_units = hp.Int('units', min_values=16, max_values=512, step=16)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10), activation='softmax')

    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  matrics=['accuracy'])

    return model

In [35]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Example hyperparameter search space
    hp_units = hp.Int('units', min_value=16, max_value=512, step=16)
    model.add(layers.Dense(units=hp_units, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model


In [37]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [39]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=5)

tuner.search(x_train,
             y_train,
             epochs=50,
             validation_split=0.2,
             callbacks=[stop_early])

Trial 30 Complete [00h 02m 06s]
val_accuracy: 0.9799166917800903

Best val_accuracy So Far: 0.9799166917800903
Total elapsed time: 00h 21m 26s


In [44]:
best_hps=tuner.get_best_hyperparameters()[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 240.



In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(48, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [46]:
h_model = tuner.hypermodel.build(best_hps)
h_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 240)               188400    
                                                                 
 dropout_1 (Dropout)         (None, 240)               0         
                                                                 
 dense_3 (Dense)             (None, 10)                2410      
                                                                 
Total params: 190810 (745.35 KB)
Trainable params: 190810 (745.35 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
h_model.fit(x_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
1500/1500 [==============================] - 10s 6ms/step - loss: 0.2780 - accuracy: 0.9187 - val_loss: 0.1325 - val_accuracy: 0.9615
Epoch 2/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.1257 - accuracy: 0.9621 - val_loss: 0.1258 - val_accuracy: 0.9623
Epoch 3/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0899 - accuracy: 0.9723 - val_loss: 0.0888 - val_accuracy: 0.9736
Epoch 4/10
1500/1500 [==============================] - 10s 6ms/step - loss: 0.0712 - accuracy: 0.9785 - val_loss: 0.0858 - val_accuracy: 0.9768
Epoch 5/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0590 - accuracy: 0.9806 - val_loss: 0.0843 - val_accuracy: 0.9754
Epoch 6/10
1500/1500 [==============================] - 8s 6ms/step - loss: 0.0482 - accuracy: 0.9847 - val_loss: 0.0800 - val_accuracy: 0.9758
Epoch 7/10
1500/1500 [==============================] - 9s 6ms/step - loss: 0.0425 - accuracy: 0.9864 - val_loss: 0.0734 - val_accurac

In [50]:
h_eval_dict = h_model.evaluate(x_test, y_test, return_dict=True)

313/313 [==============================] - 1s 3ms/step - loss: 0.0728 - accuracy: 0.9804


In [53]:
print_results(h_model, 'HYPERTUNED MODEL', 'tuned_dense_1', h_eval_dict)

NameError: name 'print_results' is not defined